Fine Tune Qwen 2.5 7b


# Install Packages

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-dme545bk/unsloth_4a0989cdd76743df90cf9742792e2b78
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-dme545bk/unsloth_4a0989cdd76743df90cf9742792e2b78
  Resolved https://github.com/unslothai/unsloth.git to commit 5f14e6fcd405db038ec099c5fe8ba4a753c8472e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# 1. Load and Prepare Training model

In [ ]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "abocide/Qwen2.5-7B-Instruct-R1-forfinance",  # << just this line
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# ensure pad token is set for packing/batching
tokenizer.pad_token = tokenizer.eos_token

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# --- Your config ---
MODEL_ID   = "abocide/Qwen2.5-7B-Instruct-R1-forfinance"  # base to fine-tune
DATA_DIR   = "/content/data_src"                          # where you'll upload JSONL
OUT_DIR    = "/content/fin_qwen2p5_qlora_unsloth"         # outputs/checkpoints

# Your sweep
LR_LIST       = [2e-4, 1e-4]
RANK_LIST     = [4, 6, 8]
DROPOUT_LIST  = [0.0, 0.1]

# Training knobs
SEED          = 42
EPOCHS_SWEEP  = 1         # light sweep
EPOCHS_FINAL  = 2         # final train
BSZ           = 2         # Unsloth is VRAM-efficient; adjust if OOM
GA            = 4
MAXLEN        = 2048      # matches notebook default
LOG_STEPS     = 10
WARMUP_STEPS  = 5         # Unsloth example uses steps rather than ratio for short runs


In [ ]:
# === W&B setup (no pip install) ===
import os
WANDB_AVAILABLE = False
try:
    import wandb  # uses preinstalled wandb if present on Colab
    # Minimal env so runs land in the right project
    os.environ.setdefault("WANDB_PROJECT", "qwen25-finetuning")
    os.environ.setdefault("WANDB_NOTEBOOK_NAME", "Fine_tube_Qwen25I7.ipynb")
    # Optional: group all sweep candidates together
    os.environ.setdefault("WANDB_RUN_GROUP", "qwen25_sweep_v1")
    # This will prompt login once per session if you’re not already authed
    wandb.login()
    WANDB_AVAILABLE = True
    print("W&B is available. Logging enabled.")
except Exception as e:
    print("W&B not available; will log only to TensorBoard. Reason:", e)


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find Fine_tube_Qwen25I7.ipynb.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rameshjeevanandan (rameshjeevanandan-university-of-birmingham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B is available. Logging enabled.


# 2. Prepare Dataset

In [ ]:
import os, shutil
from google.colab import files

DATA_DIR = "/content/data_src"
os.makedirs(DATA_DIR, exist_ok=True)

print("Upload train.jsonl, val.jsonl, test.jsonl")
uploaded = files.upload()
for fname in uploaded.keys():
    shutil.move(fname, f"{DATA_DIR}/{fname}")

!ls -l {DATA_DIR}


Upload train.jsonl, val.jsonl, test.jsonl


Saving train.jsonl to train.jsonl
Saving val.jsonl to val.jsonl
Saving test.jsonl to test.jsonl
total 1640
-rw-r--r-- 1 root root  250536 Sep  3 13:09 test.jsonl
-rw-r--r-- 1 root root 1168745 Sep  3 13:09 train.jsonl
-rw-r--r-- 1 root root  250523 Sep  3 13:09 val.jsonl


In [ ]:
from datasets import load_dataset

data_files = {
    "train": f"{DATA_DIR}/train.jsonl",
    "validation": f"{DATA_DIR}/val.jsonl",
    "test": f"{DATA_DIR}/test.jsonl",
}

raw = load_dataset("json", data_files=data_files)

def to_text(ex):
    # Expecting: {"messages":[{"role":"system/user/assistant","content":"..."}...]}
    msgs = ex["messages"]
    user = next((m["content"] for m in msgs if m.get("role")=="user"), "")
    asst = next((m["content"] for m in msgs if m.get("role")=="assistant"), "")
    return {"text": f"### Instruction:\n{user}\n\n### Response:\n{asst}"}

ds = raw.map(to_text, remove_columns=raw["train"].column_names)
train_ds, val_ds, test_ds = ds["train"], ds["validation"], ds["test"]
print("Data sizes:", len(train_ds), len(val_ds), len(test_ds))


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Data sizes: 3500 750 750


# 3. Train & Save

In [ ]:
import os, gc, math, itertools, json, re, random
import torch                             # <-- you forgot this
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments, set_seed


def get_peft(model, r=16, dropout=0.05, lora_alpha=None):
    # Add LoRA adapters via Unsloth helper (matches the original notebook style)
    peft_model = FastLanguageModel.get_peft_model(
        model,
        r = r,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
        lora_alpha = (2*r if lora_alpha is None else lora_alpha),
        lora_dropout = dropout,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = SEED,
        use_rslora = False,
        loftq_config = None,
    )
    return peft_model

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

def extract_minified_json(s: str):
    m = re.search(r'\{.*\}', s, flags=re.S)
    if not m: return None
    try:
        obj = json.loads(m.group(0))
        return json.dumps(obj, separators=(",",":"))
    except Exception:
        return None

@torch.inference_mode()
def greedy_generate_one(mdl, tok, prompt_text, max_new_tokens=128):
    inputs = tok([prompt_text], return_tensors="pt").to(mdl.device)
    out = mdl.generate(
        **inputs, max_new_tokens=max_new_tokens, do_sample=False,
        pad_token_id=tok.eos_token_id
    )
    gen = tok.decode(out[0, inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return gen.strip()

def val_score_with_generation(mdl, tok, dataset, field="text"):
    total, em = 0, 0
    for ex in dataset:
        # For EM, we assume the ground-truth JSON is in the "Response" part.
        # We look for a JSON in the example text as reference (adjust if needed).
        ref = extract_minified_json(ex[field])
        if ref is None:
            continue
        instr = ex[field].split("### Response:")[0]
        gen = greedy_generate_one(mdl, tok, instr, max_new_tokens=128)
        gen_json = extract_minified_json(gen)
        total += 1
        if gen_json is not None:
            try:
                if json.loads(gen_json) == json.loads(ref):
                    em += 1
            except Exception:
                pass

    return {"val_exact_match": (em/total if total else 0.0), "val_count": total}

set_seed(SEED)
os.makedirs(OUT_DIR, exist_ok=True)

def train_once(model, tokenizer, out_dir, lr, steps, epochs=None, **kw):
    from transformers import TrainingArguments
    from trl import SFTTrainer
    # If steps is specified, force a valid epochs value
    use_steps = steps is not None and steps > 0
    num_epochs = (epochs if (epochs is not None and epochs > 0) else (1 if use_steps else 3))

    args = TrainingArguments(
        output_dir=out_dir,
        per_device_train_batch_size=kw.get("per_device_train_batch_size", 2),
        gradient_accumulation_steps=kw.get("gradient_accumulation_steps", 4),
        learning_rate=lr,
        max_steps=(steps if use_steps else -1),   # -1 means “use epochs”
        num_train_epochs=num_epochs,              # NEVER None
        warmup_steps=kw.get("warmup_steps", 5),
        fp16=kw.get("fp16", False),               # <- changed default
        bf16=kw.get("bf16", True),                # <- changed default
        logging_steps=kw.get("logging_steps", 10),
        optim=kw.get("optim", "adamw_8bit"),
        weight_decay=kw.get("weight_decay", 0.01),
        lr_scheduler_type=kw.get("lr_scheduler_type", "linear"),
        seed=kw.get("seed", 42),
        eval_strategy=kw.get("eval_strategy", "steps"),
        eval_steps=kw.get("eval_steps", 50),
        save_strategy=kw.get("save_strategy", "no"),
        report_to=kw.get("report_to", ["tensorboard"]),
    )
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        dataset_text_field="text",
        max_seq_length=kw.get("max_seq_length", 2048),
        dataset_num_proc=kw.get("dataset_num_proc", 2),
        args=args,
    )
    trainer.train()
    return trainer

# === BAYESIAN (Optuna) SWEEP ===
!pip -q install optuna
import optuna, gc, os, torch
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# Your discrete search space (unchanged)
LR_LIST       = [2e-4, 1e-4]
RANK_LIST     = [4, 6, 8]
DROPOUT_LIST  = [0.0, 0.1]

# Compute a reasonable number of steps for the short (sweep) training
steps_per_epoch = math.ceil(len(train_ds) / (BSZ * GA))
sweep_steps = max(steps_per_epoch * EPOCHS_SWEEP, 60)  # keep your minimum

os.makedirs(OUT_DIR, exist_ok=True)
SWEEP_DIR = f"{OUT_DIR}/sweep"
os.makedirs(SWEEP_DIR, exist_ok=True)

def objective(trial):
    # 1) Sample hyperparameters
    lr      = trial.suggest_categorical("lr", LR_LIST)
    r       = trial.suggest_categorical("r", RANK_LIST)
    dropout = trial.suggest_categorical("dropout", DROPOUT_LIST)

    run_name = f"bo_lr{lr}_r{r}_drop{dropout}".replace(".", "")
    run_dir  = f"{SWEEP_DIR}/{run_name}"
    os.makedirs(run_dir, exist_ok=True)
    print(f"\n=== Optuna trial: {run_name} ===")

    # 2) Fresh base each trial
    mdl, tok = FastLanguageModel.from_pretrained(
        model_name = MODEL_ID,
        max_seq_length = MAXLEN,
        dtype = None,
        load_in_4bit = True,
    )
    tok.pad_token = tok.eos_token

    # 3) Apply LoRA for this trial
    mdl = get_peft(mdl, r=r, dropout=dropout)

    # 4) Brief training (same as your sweep)
    trainer = train_once(
        model=mdl, tokenizer=tok, out_dir=run_dir,
        lr=lr, steps=sweep_steps, epochs=None,
    )

    # 5) Validation EM (exact match)
    metrics = val_score_with_generation(trainer.model, tok, val_ds)
    em = float(metrics["val_exact_match"])
    trial.set_user_attr("val_count", int(metrics["val_count"]))
    trial.set_user_attr("out_dir", run_dir)
    print("VAL:", {"lr": lr, "r": r, "dropout": dropout, **metrics})

    # 6) Cleanup to avoid OOM
    del trainer, mdl
    cleanup()

    return em

# 7) Create & run study (maximize EM)
study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(seed=SEED),
    pruner=MedianPruner(n_warmup_steps=2),
)
# With 12 total combos, do 12 trials to cover all; or set a smaller number if you want early stopping.
study.optimize(objective, n_trials=12)

print("Best params:", study.best_params)
print("Best val EM:", study.best_value)

# 8) Final training with best params
best = study.best_params
best_lr      = best["lr"]
best_r       = best["r"]
best_dropout = best["dropout"]
print("Using best:", best)

final_dir = f"{OUT_DIR}/final_lr{best_lr}_r{best_r}_drop{best_dropout}".replace(".", "")
os.makedirs(final_dir, exist_ok=True)

mdl, tok = FastLanguageModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = MAXLEN,
    dtype = None,
    load_in_4bit = True,
)
tok.pad_token = tok.eos_token
mdl = get_peft(mdl, r=best_r, dropout=best_dropout)

final_steps = steps_per_epoch * EPOCHS_FINAL
final_trainer = train_once(
    model=mdl, tokenizer=tok, out_dir=final_dir,
    lr=best_lr, steps=final_steps, epochs=None,
)

FINAL_ADAPTER = f"{final_dir}/adapter"
final_trainer.model.save_pretrained(FINAL_ADAPTER)
tok.save_pretrained(FINAL_ADAPTER)
print("Saved final adapter:", FINAL_ADAPTER)

# 9) Test evaluation (single print)
test_metrics = val_score_with_generation(final_trainer.model, tok, test_ds)
print("\n=== TEST ===")
print(test_metrics)
print(f"Exact match: {100*test_metrics['val_exact_match']:.2f}% (n={test_metrics['val_count']})")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 22.9 MB/s eta 0:00:00


[I 2025-09-03 13:09:26,120] A new study created in memory with name: no-name-04f465ee-3d5f-4f3f-b892-d41b8f08e316



=== Optuna trial: bo_lr00001_r4_drop00 ===
==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2025.8.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


...

[I 2025-09-03 14:29:57,455] Trial 2 finished with value: 0.9986666666666667 and parameters: {'lr': 0.0001, 'r': 6, 'dropout': 0.1}. Best is trial 0 with value: 0.9986666666666667.



=== Optuna trial: bo_lr00001_r8_drop01 ===
==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,500 | Num Epochs = 1 | Total steps = 438
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,185,088 of 7,635,801,600 (0.26% trained)


Step,Training Loss,Validation Loss
50,0.625300,0.547683
100,0.448800,0.467420
150,0.458300,0.447644
200,0.411400,0.441079
250,0.430400,0.435304
300,0.462000,0.431595
350,0.405800,0.427576
400,0.404500,0.423409


In [ ]:
# ===== Leaderboard + Quick Plots in one cell =====
import pandas as pd, pathlib, time, json, math
import matplotlib.pyplot as plt

leader_dir = pathlib.Path(OUT_DIR) / "sweep"
leader_dir.mkdir(parents=True, exist_ok=True)

# Convert results (list of dicts) to a DataFrame
df = pd.DataFrame(results)

# Add some run metadata (optional)
df["model_id"] = MODEL_ID
df["timestamp"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
df["epochs_sweep"] = EPOCHS_SWEEP
df["steps_per_epoch"] = math.ceil(len(train_ds) / (BSZ * GA))
df["sweep_steps"] = df["steps_per_epoch"] * EPOCHS_SWEEP
df["batch_size"] = BSZ
df["grad_accum"] = GA
df["seed"] = SEED

# Sort by metric: highest EM first, then larger val_count
df_sorted = df.sort_values(
    by=["val_exact_match", "val_count"],
    ascending=[False, False]
).reset_index(drop=True)
df_sorted["rank"] = df_sorted.index + 1

# === Print leaderboard ===
cols_show = ["rank","run","lr","rank","dropout","val_exact_match","val_count","adapter_path"]
print("\n=== Hyperparameter Leaderboard (by Validation Exact Match) ===")
print(df_sorted[cols_show].to_string(index=False))

# === Save leaderboard ===
csv_path = leader_dir / "leaderboard.csv"
jsonl_path = leader_dir / "leaderboard.jsonl"
df_sorted.to_csv(csv_path, index=False)
with open(jsonl_path, "w") as f:
    for _, row in df_sorted.iterrows():
        f.write(json.dumps(row.to_dict()) + "\n")

print(f"\nSaved leaderboard to:\n- {csv_path}\n- {jsonl_path}")

# === Pick best config ===
best = df_sorted.iloc[0].to_dict()
print("\n=== Best config ===")
print({
    "lr": best["lr"],
    "rank": best["rank"],
    "dropout": best["dropout"],
    "val_exact_match": float(best["val_exact_match"]),
    "val_count": int(best["val_count"]),
    "adapter_path": best["adapter_path"],
})

# === Quick Plots ===

# 1) EM vs leaderboard rank
plt.figure()
plt.scatter(df_sorted["rank"], df_sorted["val_exact_match"])
plt.title("Sweep: Validation Exact Match by Leaderboard Rank")
plt.xlabel("Leaderboard Rank (1 = best)")
plt.ylabel("Validation Exact Match")
plt.grid(True)
plt.show()

# 2) EM grouped by learning rate
plt.figure()
for lr in sorted(df_sorted["lr"].unique()):
    sub = df_sorted[df_sorted["lr"] == lr].reset_index(drop=True)
    plt.plot(range(1, len(sub)+1), sub["val_exact_match"], marker="o", label=f"lr={lr}")
plt.title("Validation EM within each Learning Rate")
plt.xlabel("Run index within LR (sorted by EM)")
plt.ylabel("Validation Exact Match")
plt.legend()
plt.grid(True)
plt.show()

# 3) Boxplot of EM by LoRA rank
plt.figure()
groups = [df_sorted[df_sorted["rank"] == r]["val_exact_match"].values
          for r in sorted(df_sorted["rank"].unique())]
plt.boxplot(groups, labels=[str(r) for r in sorted(df_sorted[" rank"].unique())])
plt.title("Validation EM grouped by LoRA Rank (r)")
plt.xlabel("LoRA Rank (r)")
plt.ylabel("Validation Exact Match")
plt.grid(True)
plt.show()


:trainer = train_once(
    model=mdl, tokenizer=tok, out_dir=final_dir,
    lr=best["lr"], steps=final_steps, epochs=None,
    train_dataset=train_ds, eval_dataset=val_ds,
    run_name=f"final_lr{best['lr']}_r{best['rank']}_drop{best['dropout']}",
)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {OUT_DIR}


In [ ]:
# The trainer object was defined within the train_once function in the previous cell.
# To continue with the final training run, we need to redefine and train the trainer object here.

# Define the trainer for the final training run
final_trainer = train_once(
    model=mdl, tokenizer=tok, out_dir=final_dir,
    lr=best["lr"], steps=final_steps, epochs=None,
    train_dataset=train_ds, eval_dataset=val_ds,
    run_name=f"final_lr{best['lr']}_r{best['rank']}_drop{best['dropout']}",
)

# Train the model using the final_trainer object
trainer_stats = final_trainer.train()

Saving Model

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
# Save LoRA adapter + tokenizer
OUTPUT_DIR = "/content/lora_model"

final_trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ LoRA weights and tokenizer saved to {OUTPUT_DIR}")

In [ ]:
!zip -r lora_model.zip /content/lora_model
from google.colab import files
files.download("lora_model.zip")